# Summarize Text with Machine Learning
I will be using the extractive approach to summarize text using Machine Learning and Python. I will use the TextRank algorithm which is an extractive and unsupervised machine learning algorithm for text summarization.

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/pat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("tennis.csv")
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [3]:
df['article_text'][1]

"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours with

In [4]:
# split the sequences into the data by tokenizing them using a list
from nltk.tokenize import sent_tokenize

sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

# Use the Glove method for word representation 
An unsupervised learning algorithm developed by Stanford University to generate word integrations by aggregating the global word-to-word co-occurrence matrix from a corpus.

In [5]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

<ipython-input-5-b3ccdb22200c>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [6]:
# Create sentence vectors
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

# Finding Similarities to Summarize Text
Find similarities between the sentences, and use the cosine similarity approach for this task.

In [7]:
# Create an empty similarity matrix for this task and fill it with cosine similarities of sentences
sim_mat = np.zeros([len(sentences), len(sentences)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [8]:
# convert the sim_mat similarity matrix into the graph 
# the nodes in this graph will represent the sentences 
# the edges will represent the similarity scores between the sentences

import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

# Summarizing text

In [9]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(5):
    print("ARTICLE:")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')

ARTICLE:
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in t

# More Samples

In [10]:
text = '''The full schedule for the 2023 Major League Baseball season has been released. We already had a decent idea of the general outline, but it's always nice to get the details. 

Before even diving into the specifics on a team level, the biggest thing to discuss is the change in the shape of the schedule, so to speak. Instead of a schedule incredibly heavy on divisional matchups, it'll be much more balanced now. Check it out: 

Teams had been playing 76 in-division games. That total is now down to 52 (13 games and four series against each foe). 
Non-divisional intraleague games (for example, NL East vs. NL Central) is now a tidy 64 (32 home and 32 away) instead of 66. Each team will have seven games against four chosen opponents and six games against the other six teams fitting this criteria. 
Interleague play gets the massive increase. There used to be 20 of these games and now each team has 46. This includes a home-and-home series between "natural rivals" and then one series (seven home, seven road for each team) against the remaining 14 teams in the other league. 
Got all that? Good! Now onto the team specifics. 
'''

In [11]:
sentences2 = []
sentences2.append(sent_tokenize(text))

sentences2 = [y for x in sentences2 for y in x]

In [12]:
word_embeddings2 = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings2[word] = coefs
f.close()

clean_sentences2 = pd.Series(sentences2).str.replace("[^a-zA-Z]", " ")
clean_sentences2 = [s.lower() for s in clean_sentences2]
stop_words = stopwords.words('english')

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences2 = [remove_stopwords(r.split()) for r in clean_sentences2]

<ipython-input-12-39a7638cbae9>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences2 = pd.Series(sentences2).str.replace("[^a-zA-Z]", " ")


In [13]:
# Create sentence vectors
sentence_vectors2 = []
for i in clean_sentences2:
    if len(i) != 0:
        v = sum([word_embeddings2.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors2.append(v)

In [14]:
# Create an empty similarity matrix for this task and fill it with cosine similarities of sentences
sim_mat2 = np.zeros([len(sentences2), len(sentences2)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences2)):
    for j in range(len(sentences2)):
        if i != j:
            sim_mat2[i][j] = cosine_similarity(sentence_vectors2[i].reshape(1,100), sentence_vectors2[j].reshape(1,100))[0,0]

In [15]:
nx_graph = nx.from_numpy_array(sim_mat2)
scores2 = nx.pagerank(nx_graph)

In [16]:
ranked_sentences2 = sorted(((scores2[i],s) for i,s in enumerate(sentences2)), reverse=True)

print("SUMMARY:")

for i in range(5):
    print(ranked_sentences2[i][1])

SUMMARY:
This includes a home-and-home series between "natural rivals" and then one series (seven home, seven road for each team) against the remaining 14 teams in the other league.
Each team will have seven games against four chosen opponents and six games against the other six teams fitting this criteria.
The full schedule for the 2023 Major League Baseball season has been released.
Before even diving into the specifics on a team level, the biggest thing to discuss is the change in the shape of the schedule, so to speak.
Check it out: 

Teams had been playing 76 in-division games.


# Writing Summarization Function

In [17]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def summary(text):
    sentences = []
    sentences.append(sent_tokenize(text))
    sentences = [y for x in sentences for y in x]
    
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()

    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')

    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v) 
    
    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph) 
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("SUMMARY:")

    for i in range(5):
        print(ranked_sentences[i][1])
    
            

In [18]:
summary(text)

SUMMARY:
This includes a home-and-home series between "natural rivals" and then one series (seven home, seven road for each team) against the remaining 14 teams in the other league.
Each team will have seven games against four chosen opponents and six games against the other six teams fitting this criteria.
The full schedule for the 2023 Major League Baseball season has been released.
Before even diving into the specifics on a team level, the biggest thing to discuss is the change in the shape of the schedule, so to speak.
Check it out: 

Teams had been playing 76 in-division games.


<ipython-input-17-3474555146ba>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [19]:
text2 = '''
Opening Day for the 2023 MLB season is so close that we can taste it.
(Mmm, delicious, preposterously overpriced hot dogs and beers.)
But while we put our final bows on spring training, there's a little time left to bet on preseason win totals.
Hurry up, though. The pitch clock is ticking.
For what it's worth, betting win totals is fun but tricky and probably ill-advised. One or two injuries can derail an entire campaign, and for some teams, it all boils down to whether they can win a potentially meaningless series at the end of the season.
Nevertheless, I've talked myself into all six of these being wise investments.
The Options: Baltimore (76.5), Boston (78.5), New York (93.5), Tampa Bay (88.5), Toronto (91.5)
The Pick: New York Yankees over 93.5 wins
It's hard to fathom why the Yankees' win total is this low.
Excluding the 60-game 2020 season, the Yankees have won at least 99 games in three of the past four years. If you want to go back further than that, they've won at least 94 games in 17 out of 25 (non-pandemic) seasons dating back to 1997.
Then there's the new schedule, where instead of 76 games against the best division in baseball*, the Yankees only need to deal with their AL East rivals a combined 52 times. In place of 12 postseason-caliber games against the Rays and Blue Jays, they'll get three-game series against each of Arizona, Colorado, Miami and Washington that weren't on the schedule in 2022.
That's a pretty sweet trade-off for "Yankees over 93.5 wins" bettors.
The only way this line should be this low is if you believe the Yankees are going to be at least six wins worse than they were last season. While that certainly could be the case, what's the justification for taking that stance?
Once Carlos Rodón gets healthy, he'll be an upgrade over Jameson Taillon in what might be the best starting rotation in baseball. They didn't add any hitters via free agency, but they also didn't lose any of last year's nine leaders in plate appearances. They also figure to get much more this season out of Oswaldo Cabrera and soon-to-be rookie sensation Anthony Volpe.
Unless Aaron Judge has just a disastrous "falling back to earth" 2023 campaign, the Yankees should win at least 100 games.
'''

In [20]:
summary(text2)

SUMMARY:
The only way this line should be this low is if you believe the Yankees are going to be at least six wins worse than they were last season.
Unless Aaron Judge has just a disastrous "falling back to earth" 2023 campaign, the Yankees should win at least 100 games.
If you want to go back further than that, they've won at least 94 games in 17 out of 25 (non-pandemic) seasons dating back to 1997.
Then there's the new schedule, where instead of 76 games against the best division in baseball*, the Yankees only need to deal with their AL East rivals a combined 52 times.
They didn't add any hitters via free agency, but they also didn't lose any of last year's nine leaders in plate appearances.


<ipython-input-17-3474555146ba>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [21]:
# text is the article you wish to summarize
# n is how many sentences, top ranked to "nth" ranked
def summary_n(text,n):
    sentences = []
    sentences.append(sent_tokenize(text))
    sentences = [y for x in sentences for y in x]
    
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()

    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')

    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v) 
    
    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph) 
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    # print("Title:")
    # print(soup.title.get_text())
    # print('\n')
    
    print("SUMMARY:")

    for i in range(n):
        print(ranked_sentences[i][1])

In [22]:
summary_n(text2, 7)

SUMMARY:
The only way this line should be this low is if you believe the Yankees are going to be at least six wins worse than they were last season.
Unless Aaron Judge has just a disastrous "falling back to earth" 2023 campaign, the Yankees should win at least 100 games.
If you want to go back further than that, they've won at least 94 games in 17 out of 25 (non-pandemic) seasons dating back to 1997.
Then there's the new schedule, where instead of 76 games against the best division in baseball*, the Yankees only need to deal with their AL East rivals a combined 52 times.
They didn't add any hitters via free agency, but they also didn't lose any of last year's nine leaders in plate appearances.
But while we put our final bows on spring training, there's a little time left to bet on preseason win totals.
One or two injuries can derail an entire campaign, and for some teams, it all boils down to whether they can win a potentially meaningless series at the end of the season.


<ipython-input-21-b49082471f61>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [23]:
text_new = '''
        data-uri="archive.cms.cnn.com/_components/meta-url/instances/meta_h_3a71bf5c65e7a08f53e2be2453b538db@published"         data-uri="archive.cms.cnn.com/_components/meta-hreflang/instances/meta_h_3a71bf5c65e7a08f53e2be2453b538db@published"        data-uri="archive.cms.cnn.com/_components/meta-keywords/instances/meta_h_3a71bf5c65e7a08f53e2be2453b538db@published"      data-uri="archive.cms.cnn.com/_components/meta-authors/instances/meta_h_3a71bf5c65e7a08f53e2be2453b538db@published"     data-uri="archive.cms.cnn.com/_components/meta-robots/instances/meta_h_3a71bf5c65e7a08f53e2be2453b538db@published"     data-uri="archive.cms.cnn.com/_components/schema-org-data/instances/h_3a71bf5c65e7a08f53e2be2453b538db@published"     data-editable-end    data-editable="headLayout"    data-uri="archive.cms.cnn.com/_components/external-scripts/instances/article-with-ads-v1@published"                                         data-uri="archive.cms.cnn.com/_components/meta-site/instances/us-v1@published"        data-uri="archive.cms.cnn.com/_components/meta-icons/instances/cnn-v1@published"     data-uri="archive.cms.cnn.com/_components/ad-config/instances/cnn-article-v1@published"    data-uri="archive.cms.cnn.com/_components/analytics-optimizely/instances/default-v1@published"       data-editable-end   CNN values your feedback                                         Crime + Justice                                                     Energy + Environment                                                     Extreme Weather                                                     Space + Science                                   More                                     Crime + Justice                                       Energy + Environment                                       Extreme Weather                                       Space + Science                               Audio                       Live TV                     Settings                   Log Out                   Settings                   Log Out                     Live TV                       Audio                             US                                   International                                   Arabic                                   Español                                   US                                   International                                   Arabic                                   Español                                   Crime + Justice                                   Energy + Environment                                   Extreme Weather                                   Space + Science                               US                   Crime + Justice                       Energy + Environment                       Extreme Weather                       Space + Science                   World                   Africa                       Americas                       Asia                       Australia                       China                       Europe                       India                       Middle East                       United Kingdom                   Politics                   SCOTUS                       Congress                       Facts First                       2020 Election                   Business                   Tech                       Media                       Success                       Perspectives                       Video                   Markets                   Pre-markets                       After-Hours                       Market Movers                       Fear & Greed                       World Markets                       Investing                       Markets Now                       Before the Bell                       Nightcap                   Opinion                   Political Op-Eds                       Social Commentary                   Health                   Life, But Better                       Fitness                       Food                       Sleep                       Mindfulness                       Relationships                   Entertainment                   Movies                       Television                       Celebrity                   Tech                   Innovate                       Gadget                       Foreseeable Future                       Mission: Ahead                       Upstarts                       Work Transformed                       Innovative Cities                   Style                   Arts                       Design                       Fashion                       Architecture                       Luxury                       Beauty                       Video                   Travel                   Destinations                       Food & Drink                       Stay                       Videos                   Sports                   Pro Football                       College Football                       Basketball                       Baseball                       Soccer                       Olympics                       Hockey                   Videos                   Live TV                       Digital Studios                       CNN Films                       HLN                       TV Schedule                       TV Shows A-Z                       CNNVR                   CNN Underscored                   Electronics                       Fashion                       Beauty                       Health & Fitness                       Home                       Reviews                       Deals                       Money                       Gifts                       Travel                       Outdoors                       Pets                       CNN Store                   Coupons               More                   Photos                       Longform                       Investigations                       CNN Profiles                       CNN Leadership                       CNN Newsletters                       Work for CNN                       Police chief says no evidence Nashville shooter had specific problems or issues with school                 CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN CNN  This story contains graphic descriptions of violence.        Metro Nashville Police Chief John Drake said police have met with The Covenant School shooter’s parents and school officials and have yet to uncover any specific issues or problems in the attacker’s past.         “As of right now, we don’t have any indication there was any problems at the school or at home,” Drake told CNN on Wednesday. “We can’t confirm any type of problems at this time.”         The chief’s comments come as police have worked to answer the yet unanswerable: Why did 28-year-old Audrey Hall decide to storm into a private Christian school and murder three children and three adults?         The deadly rampage Monday morning at Covenant lasted about  14 minutes  before the shooter was shot and killed by police. The assailant killed  three 9-year-olds  as well as a custodian, a substitute teacher and the head of the school.         The massacre marked the 19th shooting at a school or university in just the past three months that left at least one person wounded, a CNN count shows. It was among  130 mass shootings this year  in the US with at least four wounded, excluding the shooting, and it was the deadliest US school shooting since last May’s  massacre in Uvalde, Texas, left 21 people dead .    Follow live updates  Austin Anthony/Reuters       Now two days from the shooting, grief-stricken Nashville residents – as well as  first lady Jill Biden  – are expected to unite at  a vigil Wednesday evening  to mourn the victims, including a 9-year-old girl described as “incredibly smart” and a 9-year-old boy noted as “unfailingly kind, gentle.”         A Nashville city councilman also said a witness told him the head of The Covenant School, Katherine Koonce, spent her last moments trying to protect the children in her care.         “The witness said Katherine Koonce was on a Zoom call, heard the shots and abruptly ended the Zoom call and left the office. The assumption from there is that she headed towards the shooter,” Councilman Russ Pulley said. He did not identify the witness.         Drake, the police chief, said he can’t confirm how Koonce died but said, “I do know she was in the hallway by herself. There was a confrontation, I’m sure. You can tell the way she is lying in the hallway.”         A college art instructor who taught the  Nashville school shooter  told CNN the attacker once had an outburst in class and appeared to be “suffering” in social media posts over the past year.         Maria Colomy, the instructor, taught the shooter,  identified by police as Audrey Hale , for two semesters in 2017 at Nossi College of Art & Design in Nashville. On the first day of class, Hale could not figure out how to set up a password, became upset and had to leave the classroom, Colomy told CNN. It was Hale’s only outburst, she said.    From The Covenant School/Covenant Presbyterian Church/Facebook/KMOV/Dieckhaus Family           Colomy described Hale as a small, quiet student and described Hale’s work as “whimsical” and “childlike.”         “I could have seen (Hale) doing children’s books for a living,” Colomy said.         Colomy followed Hale on Facebook, and for the past year Hale’s posts grieved the apparent death of a former girls’ basketball teammate, the instructor said.         “The only thing I would see (Hale) post would be about this girl,” she said.         It was around this time that Hale expressed on Facebook a desire to use he/him pronouns and the name Aiden. Police have said Hale was transgender.         “From what I saw on (Hale’s) social, (Hale) was suffering,” Colomy said.   CNN       In Monday’s attack, the killer targeted the school – which the shooter attended as a child – but it’s believed the victims were fired upon at random, police said. Police have said they do not yet know of a motive but noted the attack was closely planned.          Hale had been  under care for an emotional disorder  and legally bought seven guns in the past three years, keeping them hidden in her home from her parents, Drake said.         Tennessee does not have a “red flag” law that would allow a judge to temporarily seize guns from someone who is believed to be a threat to themselves or others. Even so, Drake said they did not know about Hale’s issues, and there is yet no evidence that Hale was believed to be a threat prior to this week.    John Amis/AP           “Law enforcement was never contacted,” he said. “She was never committed to an institution.”         The shooter is believed to have had weapons training, police spokesperson Don Aaron told CNN.         Drake also said officials are combing through a notebook of writings from Hale to learn more. Hale had written extensively in a personal notebook about the shootings, including  detailed maps  of the school, and had scouted a second possible attack location in Nashville, Drake said.         The attacker was “prepared for confrontation with law enforcement, prepared to do more harm than was actually done,” Drake said Monday.         Hale’s childhood friend told CNN on Tuesday  the killer sent her disturbing messages minutes before the attack , saying “I’m planning to die today” and it would be on the news.         Hale graduated from Nossi College of Art & Design last year, the president of the school confirmed to CNN. Hale worked as a freelance graphic designer and a part-time grocery shopper, a LinkedIn profile says.         Police have referred to Hale as a “female shooter,” and later said Hale was transgender. Hale used male pronouns on a social media profile, a spokesperson told CNN when asked to clarify.   CNN       Bodycam footage from the officers and surveillance video released by police provided a timeline of the attack and response.         Armed with three firearms, the shooter entered the school by firing at glass doors and climbing through to get inside, then walked down the hallways and pointed an assault-style weapon, surveillance video released by Metro Nashville Police shows.         The first call about the shooting came in at 10:13 a.m., and police rushed to the school, arriving at 10:24 a.m., according to the police chief.         Police on Tuesday released body-camera footage from the two officers who opened fire on the shooter after rushing into the school on Monday. The footage shows a group of five officers entered the school amid wailing fire alarms and immediately went into several empty classrooms rooms to look for the suspect.    John Amis/AP           As they cleared the rooms, officers heard gunfire from upstairs and rushed up to the second floor. An officer armed with an assault-style rifle fired multiple times at a person near a large window, who dropped to the ground, the video shows.         Another officer then appeared to shoot the person on the ground four times with a handgun, yelling, “Stop moving!” The officers then approached the person, moved a gun away and radioed, “Suspect down! Suspect down!”         The shooter was dead at 10:27 a.m., police said.         As a private school operated by a church, there was no school resource officer assigned by the city to guard the school, according to Aaron, the police spokesman.         Asked about the roughly 11-minute gap between when police received the first call of an active shooter and when officers arrived at the school, the police chief told reporters, “From what I’ve seen, I don’t have a particular problem with it. But we always want to get better. We always want to get there in 2 or 3 minutes, and so there’s a lot of things that could have happened – traffic was locked down, etc.”         The quick law enforcement response in Nashville stands in notable contrast with the delay in Uvalde of more than an hour before authorities confronted and killed the gunman – a lag that revived a  nationwide conversation  about the use of force during shootings  in public places , especially schools.   From The Covenant School/Covenant Presbyterian Church/Facebook/KMOV/Dieckhaus Family       The victims of the shooting included three 9-year-old students: Evelyn Dieckhaus, William Kinney and Hallie Scruggs, the  daughter of lead church pastor Chad Scruggs .          Hallie’s aunt Kara Arnold said the 9-year-old had “a love for life that kept her smiling and running and jumping and playing and always on the go.”         “To watch her these past 9 years has been a gift and a privilege,”  Arnold wrote on Facebook . “I have often longed for a daughter and Hallie embodied all of those things I’d want in my own little girl.”         She said Hallie and her son Chip were almost the same age and were very close.         “When they were together no one else existed and we would rarely know where they were or what they were up to,” she wrote, adding that they were usually involved in some sort of mischief.         “She was incredibly smart, feisty enough to keep up with her 3 brothers and my 4 boys, a love for life that kept her smiling and running and jumping and playing and always on the go,” Arnold wrote.         Arnold told CNN that she is on her way to Nashville to be with her brother and sister-in-law.    Mark Zaleski/USA Today Network/Reuters           “I don’t have the words to address this tragedy. I’m in shock and my boys and Adam and I can’t even comprehend that she’s gone,” she wrote. “We are grieving so deeply. And for my brother and sister in law and my nephews who are living the most unspeakable nightmare. We hurt with you and wish we could take your hurt away.”         William, another of the children killed, had an “unflappable spirit,” friends of the Kinney family shared on GoFundMe.         “He was unfailingly kind, gentle when the situation called for it, quick to laugh, and always inclusive of others,” they wrote.         Will had two sisters and enjoyed spending time with his family and friends. “Our hearts are broken for his family as they try to find their way forward,” the friends said.         Evelyn’s family appreciates all the love and support they’ve received and are asking for space as they grieve, according to a family statement obtained by CNN affiliate KMOV.         “Our hearts are completely broken. We cannot believe this has happened,” the statement said. “Evelyn was a shining light in this world.”         The adults killed in addition to Koonce, 60, were Cynthia Peak, 61, a substitute teacher, and Mike Hill, a 61-year-old custodian, police said.         Peak was best friends with Tennessee First Lady Maria Lee and was supposed to go to the Lees’ home for dinner Monday evening, Tennessee Gov. Bill Lee said in a video statement Tuesday.          “Maria woke up this morning without one of her best friends, Cindy Peak,” the governor said. “Cindy and Maria and Katherine Koonce were all teachers at the same school and have been family friends for decades.”         Hill was a father of seven children and grandfather to 14 who loved to cook and spend time with his family, his family  said  in a statement obtained by CNN affiliate  WSMV . Evelyn’s family  called her  “a shining light in this world.”         The city has set up a  fund  to help support the survivors of the shooting, Mayor John Cooper said.      CNN’s Melissa Alonso, Mark Morales, Amara Walker, Dianne Gallagher, Tina Burnside, Amanda Jackson, Sara Smart, David Williams, Jillian Sykes, Jamiel Lynch, Holly Yan and Michelle Krupa contributed to this report.                                       Settings                   Log Out                 Live TV               Audio               US                   Crime + Justice                       Energy + Environment                       Extreme Weather                       Space + Science                   World                   Africa                       Americas                       Asia                       Australia                       China                       Europe                       India                       Middle East                       United Kingdom                   Politics                   The Biden Presidency                       Facts First                       US Elections                   Business                   Tech                       Media                       Success                       Perspectives                       Videos                   Markets                   Pre-markets                       After-Hours                       Market Movers                       Fear & Greed                       World Markets                       Investing                       Markets Now                       Before the Bell                       Nightcap                   Opinion                   Political Op-Eds                       Social Commentary                   Health                   Life, But Better                       Fitness                       Food                       Sleep                       Mindfulness                       Relationships                   Entertainment                   Movies                       Television                       Celebrity                   Tech                   Innovate                       Gadget                       Foreseeable Future                       Mission: Ahead                       Upstarts                       Work Transformed                       Innovative Cities                   Style                   Arts                       Design                       Fashion                       Architecture                       Luxury                       Beauty                       Video                   Travel                   Destinations                       Food & Drink                       Stay                       News                       Videos                   Sports                   Pro Football                       College Football                       Basketball                       Baseball                       Soccer                       Olympics                       Hockey                   Videos                   Live TV                       Digital Studios                       CNN Films                       HLN                       TV Schedule                       TV Shows A-Z                       CNNVR                   Audio               CNN Underscored                   Electronics                       Fashion                       Beauty                       Health & Fitness                       Home                       Reviews                       Deals                       Money                       Gifts                       Travel                       Outdoors                       Pets                       CNN Store                   Coupons               Weather                   Climate                       Storm Tracker                       Wildfire Tracker                       Video                   More                   Photos                       Longform                       Investigations                       CNN Profiles                       CNN Leadership                       CNN Newsletters                       Work for CNN                           Audio                       Live TV                           Settings                   Log Out                   Terms of Use                   Privacy Policy                    Ad Choices                   Accessibility & CC                   About                   Newsletters                   Transcripts         © 2023 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved.   CNN Sans ™ & © 2016 Cable News Network. '
'''

In [24]:
summary_n(text_new,5)

<ipython-input-21-b49082471f61>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


SUMMARY:
“We can’t confirm any type of problems at this time.”         The chief’s comments come as police have worked to answer the yet unanswerable: Why did 28-year-old Audrey Hall decide to storm into a private Christian school and murder three children and three adults?
Follow live updates  Austin Anthony/Reuters       Now two days from the shooting, grief-stricken Nashville residents – as well as  first lady Jill Biden  – are expected to unite at  a vigil Wednesday evening  to mourn the victims, including a 9-year-old girl described as “incredibly smart” and a 9-year-old boy noted as “unfailingly kind, gentle.”         A Nashville city councilman also said a witness told him the head of The Covenant School, Katherine Koonce, spent her last moments trying to protect the children in her care.
Metro Nashville Police Chief John Drake said police have met with The Covenant School shooter’s parents and school officials and have yet to uncover any specific issues or problems in the attac